In [ ]:
# ! pip install pydub
# ! pip install -U datasets
# ! pip install -U accelerate
# ! pip install -U transformers

In [1]:
! pip install numpy==1.22.4

In [ ]:
# how to print the nupy version:
import importlib
import numpy as np

# Make some changes to the numpy module or its files

# Now reload the module
# importlib.reload(np)

print(np.__version__)

In [ ]:
!pip show numpy

In [2]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('roberta-large')
model = AutoModel.from_pretrained('roberta-large')


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
!pwd
# !curl -o multinli_1.0.zip https://cims.nyu.edu/~sbowman/multinli/multinli_1.0.zip
# !unzip multinli_1.0.zip
!ls

/kaggle/working


In [4]:
dir_data = "/kaggle/working/multinli_1.0"
dir_data = '/kaggle/input/bonus2-lora/multinli_1.0'
import pandas as pd

# Path to the dataset file
file_path = f'{dir_data}/multinli_1.0_train.jsonl'

# Load the dataset
data = pd.read_json(file_path, lines=True)
# print(data)

In [5]:
print(type(data))
print(data.info())
print(data.columns)
# print(data.head(1))
# print(data)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392702 entries, 0 to 392701
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   annotator_labels        392702 non-null  object
 1   genre                   392702 non-null  object
 2   gold_label              392702 non-null  object
 3   pairID                  392702 non-null  object
 4   promptID                392702 non-null  int64 
 5   sentence1               392702 non-null  object
 6   sentence1_binary_parse  392702 non-null  object
 7   sentence1_parse         392702 non-null  object
 8   sentence2               392702 non-null  object
 9   sentence2_binary_parse  392702 non-null  object
 10  sentence2_parse         392702 non-null  object
dtypes: int64(1), object(10)
memory usage: 33.0+ MB
None
Index(['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID',
       'sentence1', 'sentence1_bi

## fine-tuning

In [6]:
# load the datset:
from datasets import load_dataset
from transformers import AutoTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from datasets import ClassLabel, Value, load_metric

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-large')

# Function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'],
                      truncation=True)

# Load and tokenize the dataset

# dataset = load_dataset('multi_nli', split='train')

dir_data = "/kaggle/working/multinli_1.0"
dir_data = '/kaggle/input/bonus2-lora/multinli_1.0'

# Path to the local files
train_file = f'{dir_data}/multinli_1.0_train.jsonl'
matched_validation_file = f'{dir_data}/multinli_1.0_dev_matched.jsonl'
mismatched_validation_file = f'{dir_data}/multinli_1.0_dev_mismatched.jsonl'

# Load the datasets
train_dataset = load_dataset('json', data_files={'train': train_file}, split='train')
matched_validation_dataset = load_dataset('json', data_files={'validation_matched': matched_validation_file}, split='validation_matched')
mismatched_validation_dataset = load_dataset('json', data_files={'validation_mismatched': mismatched_validation_file}, split='validation_mismatched')
# the train data is too big to load, so we load the first 10000 rows:
# train_dataset = train_dataset.select(range(10000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9728a3cffb4e95a6/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0584a5e343aa3bd7/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-fbbbb5ea9042b6cd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


In [7]:
train_dataset

Dataset({
    features: ['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse'],
    num_rows: 392702
})

In [8]:
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_matched_validation_dataset = matched_validation_dataset.map(tokenize_function, batched=True)
tokenized_mismatched_validation_dataset = mismatched_validation_dataset.map(tokenize_function, batched=True)

  0%|          | 0/393 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [9]:
tkn_trn_dta = tokenized_train_dataset
tkn_mch_vld_dta = tokenized_matched_validation_dataset
tkn_msmch_vld_dta = tokenized_mismatched_validation_dataset

In [10]:
print(type(tkn_trn_dta))
print(tkn_mch_vld_dta)
print(tkn_trn_dta)
print(tkn_trn_dta.select([0]))
print(tkn_trn_dta.select([0]).to_dict()['gold_label'])

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse', 'input_ids', 'attention_mask'],
    num_rows: 10000
})
Dataset({
    features: ['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse', 'input_ids', 'attention_mask'],
    num_rows: 392702
})
Dataset({
    features: ['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse', 'input_ids', 'attention_mask'],
    num_rows: 1
})
['neutral']


In [11]:
print(tokenized_matched_validation_dataset.column_names)
print(tokenized_mismatched_validation_dataset.column_names)

['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse', 'input_ids', 'attention_mask']
['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse', 'input_ids', 'attention_mask']


In [12]:
label_dict = {'neutral': 0, 'contradiction': 1, 'entailment': 2}


In [13]:
# convert the labels to indices for all the datsets:
tokenized_train_dataset = tokenized_train_dataset.map(lambda example: {'gold_label': label_dict[example['gold_label']]})

  0%|          | 0/392702 [00:00<?, ?ex/s]

In [14]:
print(tokenized_train_dataset[0])
print(type(tokenized_matched_validation_dataset['gold_label']))

{'annotator_labels': ['neutral'], 'genre': 'government', 'gold_label': 0, 'pairID': '31193n', 'promptID': '31193', 'sentence1': 'Conceptually cream skimming has two basic dimensions - product and geography.', 'sentence1_binary_parse': '( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )', 'sentence1_parse': '(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))', 'sentence2': 'Product and geography are what make cream skimming work. ', 'sentence2_binary_parse': '( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )', 'sentence2_parse': '(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))', 'input_ids': [0, 9157, 16771, 13851, 6353, 2972, 

In [15]:
# Step 1: Identify unique labels
unique_labels = set(tokenized_mismatched_validation_dataset['gold_label'])
label_to_index_val_mis = {label: idx for idx, label in enumerate(unique_labels)}
print(label_to_index_val_mis)
unique_labels = set(tokenized_matched_validation_dataset['gold_label'])
label_to_index_val_mtch = {label: idx for idx, label in enumerate(unique_labels)}
label_to_index_val_mtch

{'contradiction': 0, 'neutral': 1, 'entailment': 2, '-': 3}


{'contradiction': 0, 'neutral': 1, '-': 2, 'entailment': 3}

In [16]:
tokenized_matched_validation_dataset = tokenized_matched_validation_dataset.filter(lambda example: example['gold_label'] != '-')
tokenized_mismatched_validation_dataset = tokenized_mismatched_validation_dataset.filter(lambda example: example['gold_label'] != '-')


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [17]:
tokenized_matched_validation_dataset = tokenized_matched_validation_dataset.map(lambda example: {'gold_label': label_dict[example['gold_label']]})
tokenized_mismatched_validation_dataset = tokenized_mismatched_validation_dataset.map(lambda example: {'gold_label': label_dict[example['gold_label']]})

  0%|          | 0/9815 [00:00<?, ?ex/s]

  0%|          | 0/9832 [00:00<?, ?ex/s]

In [18]:
# see the tokenized dataset:
print(type(tokenized_train_dataset[0]))
print(tokenized_train_dataset[0].keys())
print(tokenized_train_dataset.column_names)


<class 'dict'>
dict_keys(['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse', 'input_ids', 'attention_mask'])
['annotator_labels', 'genre', 'gold_label', 'pairID', 'promptID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse', 'input_ids', 'attention_mask']


In [19]:
# run this if and only if sth failed.
# tokenized_train_dataset = tkn_trn_dta
# tokenized_matched_validation_dataset = tkn_mch_vld_dta
# tokenized_mismatched_validation_dataset = tkn_msmch_vld_dta

In [128]:
# # Format our dataset to outputs torch.Tensor
# tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "pairID", "promptID", "genre", "label1", "label2", "label3", "label4", "label5"])
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# tokenized_datasets.set_format('torch')



columns_to_remove = ["annotator_labels", "sentence1", "pairID", "promptID", "genre",
                     "sentence1_binary_parse", "sentence1_parse", "sentence2", "sentence2_binary_parse", "sentence2_parse"]

tokenized_train_dataset = tokenized_train_dataset.remove_columns(columns_to_remove)
tokenized_train_dataset = tokenized_train_dataset.rename_column("gold_label", "labels")
tokenized_train_dataset.set_format('torch')

tokenized_matched_validation_dataset = tokenized_matched_validation_dataset.remove_columns(columns_to_remove)
tokenized_matched_validation_dataset = tokenized_matched_validation_dataset.rename_column("gold_label", "labels")
tokenized_matched_validation_dataset.set_format('torch')

tokenized_mismatched_validation_dataset = tokenized_mismatched_validation_dataset.remove_columns(columns_to_remove)
tokenized_mismatched_validation_dataset = tokenized_mismatched_validation_dataset.rename_column("gold_label", "labels")
tokenized_mismatched_validation_dataset.set_format('torch')

In [ ]:
# # Split the dataset into training and validation
tokenized_train_dataset = tokenized_train_dataset.shuffle(seed=42).select(range(2000,3000))  # Use a smaller subset for training
tokenized_matched_validation_dataset = tokenized_matched_validation_dataset.shuffle(seed=42).select(range(1000, 1100))  # Use a different subset for evaluation

In [ ]:
# # remove the column 'annotator_labels' from the datasets:
# columns_to_remove = ['annotator_labels']
# tokenized_train_dataset = tokenized_train_dataset.remove_columns(columns_to_remove)
# tokenized_matched_validation_dataset = tokenized_matched_validation_dataset.remove_columns(columns_to_remove)
# tokenized_mismatched_validation_dataset = tokenized_mismatched_validation_dataset.remove_columns(columns_to_remove)


In [131]:
print(type(tokenized_train_dataset))
print(tokenized_train_dataset.format)
print(tokenized_train_dataset)
print(tokenized_train_dataset.select([0]))
print(tokenized_train_dataset.select([0]).to_dict()['labels'])

<class 'datasets.arrow_dataset.Dataset'>
{'type': 'torch', 'format_kwargs': {}, 'columns': ['labels', 'input_ids', 'attention_mask'], 'output_all_columns': False}
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1000
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1
})
[0]


In [132]:
# find the unique labels:
labels = set()
for idx in range(100):
    labels.add(tokenized_train_dataset.select([idx]).to_dict()['labels'][0])

In [133]:
print(labels)
label_dict = {'neutral': 0, 'contradiction': 1, 'entailment': 2}


{0, 1, 2}


In [134]:
# tokenized_train_dataset.reset_format()
# print(tokenized_train_dataset[0])
# tokenized_train_dataset.set_format('torch')
# print(tokenized_train_dataset[0])

In [135]:
# for example in tokenized_train_dataset:
#     print(example['labels'])
    
# see the tokenized dataset:
print(type(tokenized_train_dataset))
print(tokenized_train_dataset.format)
print(tokenized_train_dataset[0])
print(tokenized_train_dataset.column_names)

<class 'datasets.arrow_dataset.Dataset'>
{'type': 'torch', 'format_kwargs': {}, 'columns': ['labels', 'input_ids', 'attention_mask'], 'output_all_columns': False}
{'labels': tensor(0), 'input_ids': tensor([    0,   250, 22040,     9, 16045, 20536,  2640,  2074,  1595,    30,
          982,     8,  1822,    81,     5,    94,   891,     9,   107,   492,
         4674,     5,   476,     7,   146,  1041,   582,    13, 13430,  6848,
           50, 10946, 15789,    19,    49,  1159,     4,     2,     2,   133,
         1041,    32,    45,   182,  1372,    59,     5,    92,  4361,    14,
         4674,    33,     4,     2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])}
['labels', 'input_ids', 'attention_mask']


In [136]:
print(";")

;


In [137]:
import os
# os.environ["WANDB_DISABLED"] = "true"
# os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

In [138]:
# Load the model
model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [139]:
print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [171]:
from torch.nn.utils.rnn import pad_sequence
import torch

class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, examples):
        # Assuming 'examples' is a batch from your dataset
        input_ids = [example['input_ids'].clone().detach() for example in examples]
        attention_masks = [example['attention_mask'].clone().detach() for example in examples]
        labels = torch.tensor([example['labels'] for example in examples])
        print(labels, labels.dtype)
        input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_masks_padded = pad_sequence(attention_masks, batch_first=True, padding_value=0)

        return {
            'input_ids': input_ids_padded,
            'attention_mask': attention_masks_padded,
            'labels': labels
        }

    
data_collator = CustomDataCollator(tokenizer)

In [141]:
print("Train dataset size:", len(tokenized_train_dataset))
print("Example data:", tokenized_train_dataset[0])


Train dataset size: 1000
Example data: {'labels': tensor(0), 'input_ids': tensor([    0,   250, 22040,     9, 16045, 20536,  2640,  2074,  1595,    30,
          982,     8,  1822,    81,     5,    94,   891,     9,   107,   492,
         4674,     5,   476,     7,   146,  1041,   582,    13, 13430,  6848,
           50, 10946, 15789,    19,    49,  1159,     4,     2,     2,   133,
         1041,    32,    45,   182,  1372,    59,     5,    92,  4361,    14,
         4674,    33,     4,     2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])}


In [142]:
from sklearn.metrics import accuracy_score

# def compute_metrics(eval_pred):
#     print(eval_pred)
#     logits, labels = eval_pred
#     predictions = logits.argmax(axis=-1)
#     return {'accuracy': accuracy_score(labels, predictions)}

def compute_metrics(pred):
#     print(eval_pred)
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}


In [143]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=4,   
    gradient_accumulation_steps=2,
    eval_steps=20,
    evaluation_strategy = 'steps',
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',           
    logging_steps=20,
    report_to='none',
    do_train=True,
    do_eval=True,
    do_predict=True,
#     fp16=True,
)

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_matched_validation_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics 
)

In [144]:
# free gpu memory:
import gc
gc.collect()
torch.cuda.empty_cache()
# free gpu memory:
import gc
gc.collect()
torch.cuda.empty_cache()


In [145]:
# calculate the time:
import time
start_time = time.time()

# Train the model
trainer.train()

# calculate the time:
print("--- %s seconds ---" % (time.time() - start_time))

# You can save the model and tokenizer
trainer.save_model("./multi_nli_roberta")
tokenizer.save_pretrained("./multi_nli_roberta")

Step,Training Loss,Validation Loss,Accuracy
20,1.129600,1.097398,0.360000
40,1.096700,1.104369,0.350000
60,1.103000,1.104729,0.310000
80,1.111700,1.095781,0.360000
100,1.083100,1.072380,0.450000
120,1.048400,0.978953,0.510000
140,0.977800,1.115750,0.480000
160,0.969500,0.844807,0.650000
180,0.827200,0.827574,0.660000
200,0.783700,0.718026,0.740000


--- 122.53531670570374 seconds ---


('./multi_nli_roberta/tokenizer_config.json',
 './multi_nli_roberta/special_tokens_map.json',
 './multi_nli_roberta/vocab.json',
 './multi_nli_roberta/merges.txt',
 './multi_nli_roberta/added_tokens.json',
 './multi_nli_roberta/tokenizer.json')

In [ ]:
# free gpu memory:
import gc
gc.collect()
torch.cuda.empty_cache()
# free gpu memory:
import gc
gc.collect()
torch.cuda.empty_cache()

In [37]:
# # calculate the time:
# import time
# start_time = time.time()

# # Evaluate the model
# eval_result = trainer.evaluate(tokenized_matched_validation_dataset)

# # evaluate the model  on the mismatched validation dataset:
# trainer.evaluate(tokenized_mismatched_validation_dataset)

# # calculate the time:
# print("--- %s seconds ---" % (time.time() - start_time))


In [38]:
# free gpu memory:
import gc
gc.collect()
torch.cuda.empty_cache()
# free gpu memory:
import gc
gc.collect()
torch.cuda.empty_cache()

## LoRA

In [39]:
!pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/8b/1b/aee2a330d050c493642d59ba6af51f3910cb138ea48ede228c84c204a5af/peft-0.7.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 7.7 MB/s eta 0:00:00


In [172]:
# Fine-tuning using the LoRA approach:
from peft import LoraConfig
from transformers import RobertaForSequenceClassification

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],  # Specify which modules to apply LoRA
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],  # Specify modules to keep trainable
)

from peft import get_peft_model

model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=3)
model = get_peft_model(model, lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [199]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(model)

# After applying LoRA modifications to your model
num_trainable_params = count_trainable_parameters(model)
print(f"Number of trainable parameters: {num_trainable_params}")

PeftModel(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
              

In [184]:
# data_collator2 = CustomDataCollator(tokenizer)
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print("Training dataset size:", len(tokenized_train_dataset))
print("Mismatched validation dataset size:", len(tokenized_mismatched_validation_dataset))

Training dataset size: 1000
Mismatched validation dataset size: 9832


In [197]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=1,   
#     gradient_accumulation_steps=2,
    eval_steps=20,
    evaluation_strategy = 'steps',
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',           
    logging_steps=20,
    report_to='none',
    do_train=True,
    do_eval=True,
    do_predict=True,
#     fp16=True,
)

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
#     train_dataset=tokenized_train_dataset,
    train_dataset=tokenized_mismatched_validation_dataset,
#     eval_dataset=tokenized_matched_validation_dataset,
#     data_collator=data_collator,
    compute_metrics=compute_metrics 
)

In [ ]:
# Calculate time:
import time
start_time = time.time()

# Train the model
trainer.train()

# Evaluate the model
eval_result = trainer.evaluate()

# evaluate the model  on the mismatched validation dataset:
trainer.evaluate(tokenized_mismatched_validation_dataset)

# calculate the time:
print("--- %s seconds ---" % (time.time() - start_time))

# You can save the model and tokenizer
trainer.save_model("./multi_nli_roberta_lora")
tokenizer.save_pretrained("./multi_nli_roberta_lora")